In [ ]:
import pandas as pd

#Apertura del dataset
df = pd.read_json('../datasets/shelflife.json')
df.head(5)

In [ ]:
#Preparando el dataset para hacer la clasificacion 
df_clasificado = df.copy()
columnas_usar = ['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']
df_clasificado = df_clasificado[columnas_usar]

#Rellenar los valores null con ceros
columnas_numericas = ['shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']
df_clasificado[columnas_usar] = df_clasificado[columnas_usar].fillna(0)

df_clasificado.head(15)

In [ ]:
# Clasificacion en perecedero = 1 o no_perecedero = 0

def clasificar_perecedero(fila):
    """
    Clasifica un producto como perecedero (1) o no perecedero (0) según su categoría y vida útil.

    La función evalúa si un producto pertenece a una categoría no perecedera o si requiere refrigeración/
    congelación. También se considera perecedero si su vida útil en despensa es muy corta.

    Args:
        fila (pd.Series): Fila del DataFrame con información del producto. Debe contener las siguientes columnas:
            - 'category_off' (str): Categoría del producto.
            - 'shelf_life_fridge_days' (int): Días de vida útil si se guarda en refrigeración.
            - 'shelf_life_freezer_days' (int): Días de vida útil si se guarda en congelación.
            - 'shelf_life_pantry_days' (int): Días de vida útil si se guarda en despensa.

    Returns:
        int: 1 si el producto es perecedero, 0 si no lo es.
    """
    
    categorias_no_perecederas = [
        'juice-box', 'canned-fish', 'beans-dry', 'sugar-white', 'salt',
        'soft-drinks', 'tea-bags', 'oil-vegetable', 'chips', 'rice-white-dry',
        'pasta-dry', 'canned-tomato', 'canned-meat', 'cookies', 'cereal-box',
        'coffee-ground', 'milk-uht', 'bottled-water', 'salsa-jarred',
        'toothpaste', 'shampoo', 'household-detergent'
    ]

    # Si pertenece a una categoría no perecedera 0
    if fila['category_off'] in categorias_no_perecederas:
        return 0
    # Si requiere de estar en un ambiente frio es 1
    elif (fila['shelf_life_fridge_days'] > 0 or fila['shelf_life_freezer_days'] > 0):
        return 1  # Perecedero (Se beneficia del frío)

    #El producto no va en frío pero su vida útil en despensa es muy corta
    elif fila['shelf_life_pantry_days'] < 20:
        return 1  # Perecedero por vida util muy corta
    else:
        return 0

# Aplicar la clasificación
df_clasificado['perecedero'] = df.apply(clasificar_perecedero, axis=1)


df_clasificado.tail(100)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

df_clasificado = pd.read_csv('../datasets/productos_clasificados.csv')

X = df_clasificado[['category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']]
y = df_clasificado['perecedero']

# Conversion de categorias a numeros para que el modelo pueda ejecutar el algoritmo
encoder = LabelEncoder()
X['category_off'] = encoder.fit_transform(X['category_off'])

# Separar los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Entrenar modelo
modelo = RandomForestClassifier(n_estimators=500, random_state=42)
modelo.fit(X_train, y_train)

# Evaluar modelo
score = modelo.score(X_test, y_test)
print(f"Precisión: {score:.2f}")


In [ ]:
#Matriz de confusion
from sklearn.metrics import confusion_matrix, classification_report

y_pred = modelo.predict(X_test)

#Generacion de la matriz
matriz_confusion = confusion_matrix(y_test, y_pred)
print('Matriz de confusin :')
print(matriz_confusion)

In [ ]:
#Haciendo pruebas del modelo con datos nnunca vistos (completamnete nuevos)

# Los nuevos productos
productos_nuevos = [
    ["Plátano tabasco 1 kg", "fruit-fresh", 5, 0, 0],
    ["Tomate saladet 1 kg", "vegetable-fresh", 4, 0, 0],
    ["Limón agrio 1 kg", "fruit-fresh", 7, 0, 0],
    ["Aguacate hass 1 kg", "fruit-fresh", 3, 0, 0],
    ["Pan dulce 1 pza", "bakery-fresh", 3, 0, 0],
    ["Tortillas de maíz 1 kg", "tortilla", 5, 10, 30],
    ["Jamón de pavo 250 g", "meat-processed", 0, 10, 60],
    ["Yogur de fresa 1L", "yogurt", 0, 15, 0],
    ["Queso panela 400 g", "cheese-soft", 0, 14, 90],
    ["Refresco de cola 600 ml", "soda", 365, 0, 0],
    ["Aceite vegetal 1L", "oil-vegetable", 730, 0, 0],
    ["Café molido 250 g", "coffee-ground", 540, 0, 0],
    ["Sal yodada 1 kg", "salt", 1825, 0, 0],
    ["Harina de trigo 1 kg", "flour", 365, 0, 0],
    ["Frijol negro 1 kg", "beans-dry", 365, 0, 0],
    ["Pollo entero crudo 1 kg", "meat-raw", 0, 5, 180],
    ["Pescado mojarra 1 kg", "fish-fresh", 0, 3, 150],
    ["Atún enlatado 140 g", "canned-fish", 720, 0, 0],
    ["Cereal de maíz 500 g", "cereal", 365, 0, 0],
    ["Mantequilla 200 g", "butter", 0, 90, 180],
]

df_nuevos = pd.DataFrame(productos_nuevos, columns=[
    "product_name", "category_off", "shelf_life_pantry_days",
    "shelf_life_fridge_days", "shelf_life_freezer_days"
])

In [ ]:
X_nuevos = df_nuevos[['category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']]

X_nuevos['category_off'] = encoder.fit_transform(X_nuevos['category_off'])

predicciones = modelo.predict(X_nuevos)
df_nuevos['perecedero_prediccion'] = predicciones

df_nuevos.head(10)

In [ ]:
# Agregar las etiquetas reales
etiquetas_reales = [1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0,0,1]
df_nuevos['perecedero_real'] = etiquetas_reales

# Comparar
df_nuevos['acierto'] = df_nuevos['perecedero_prediccion'] == df_nuevos['perecedero_real']

# Calcular precision con productos nuevos
precision_nuevos = df_nuevos['acierto'].mean()
print(f"Precisión con productos nuevos: {precision_nuevos:.2f}")

df_nuevos_guardar = df_nuevos[['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days', 'perecedero_real']]
ultimo_indice = pd.read_csv('../datasets/productos_clasificados.csv', usecols=['Unnamed: 0']).iloc[-1, 0]
df_nuevos_guardar.index = range(ultimo_indice + 1, ultimo_indice + 1 + len(df_nuevos))
df_nuevos_guardar.to_csv('../datasets/productos_clasificados.csv', mode='a', header=False, index=True)

df_nuevos.head(20)



In [ ]:
def clasificador_nuevo_producto(nombre:str, categoria:str, 
                                vida_normal:int, vida_refrigeracion:int, vida_congelacion:int):
    
    """
    Clasifica un nuevo producto como perecedero o no perecedero utilizando un modelo de machine learning
    entrenado anteriormente, y actualiza el dataset con el nuevo resultado.

    Esta función recibe la informacion basica de un producto (nombre, categoría y vida útil en distintos
    ambientes), la transforma según el codificador de categorías, obtiene una predicción del modelo,
    y finalmente guarda el nuevo registro en el archivo de productos clasificados.

    Args:
        nombre (str): Nombre del producto.
        categoria (str): Categoría del producto.
        vida_normal (int): Días de vida útil del producto almacenado a temperatura ambiente.
        vida_refrigeracion (int): Días de vida útil del producto si se conserva en refrigeración.
        vida_congelacion (int): Días de vida útil del producto si se conserva congelado.

    Returns:
        str: Mensaje indicando el resultado de la clasificación.
             Por ejemplo: `"El producto 'Manzana' es de tipo: perecedero"`
    """
    
    ruta = '../datasets/productos_clasificados.csv'
    
    datos_producto = pd.DataFrame({
            'category_off': [categoria],  
            'shelf_life_pantry_days': [vida_normal],
            'shelf_life_fridge_days': [vida_refrigeracion],
            'shelf_life_freezer_days': [vida_congelacion]
        })

        
    datos_producto['category_off'] = encoder.fit_transform(datos_producto['category_off'])
    prediccion = modelo.predict(datos_producto)
    
    ultimo_indice = pd.read_csv(ruta, usecols=['Unnamed: 0']).iloc[-1, 0]
    
    datos_producto['perecedero'] = prediccion
    datos_producto['category_off'] = categoria
    datos_producto['product_name'] = nombre


    datos_producto = datos_producto[['product_name', 'category_off',
                                     'shelf_life_pantry_days', 'shelf_life_fridge_days',
                                     'shelf_life_freezer_days', 'perecedero']]
    
    datos_producto.index = [ultimo_indice + 1]


    # Guardar solo la nueva fila
    datos_producto.to_csv(ruta, mode='a', header=False, index=True)
        
    resultado = 'perecedero' if prediccion[0] == 1 else 'no perecedero'

    return f"El producto '{nombre}' es de tipo: {resultado}"

In [ ]:
prueba_mango = clasificador_nuevo_producto(
    nombre="Mango Ataulfo 1kg",
    categoria="fruit-fresh",
    vida_normal=5,
    vida_refrigeracion=10,
    vida_congelacion=180
)
print(prueba_mango) 

In [ ]:
prueba_manzana = clasificador_nuevo_producto(
    nombre="Manzana Gala en Bolsa 1kg",
    categoria="fruit-fresh",
    vida_normal=14,
    vida_refrigeracion=45,
    vida_congelacion=200
)
print(prueba_manzana)

In [ ]:
prueba_pasta = clasificador_nuevo_producto(
    nombre="Pasta de Espagueti 500g",
    categoria="pasta-dry",
    vida_normal=730,
    vida_refrigeracion=0,
    vida_congelacion=0
)
print(prueba_pasta)

In [ ]:
prueba_galletas = clasificador_nuevo_producto(
    nombre="Galletas Saladas en Caja 300g",
    categoria="cookies",
    vida_normal=270,
    vida_refrigeracion=0,
    vida_congelacion=0
)
print(prueba_galletas)

In [ ]:
prueba_vino = clasificador_nuevo_producto(
    nombre="Vino Tinto Cabernet Sauvignon 750ml",
    categoria="beverage-alcoholic", # La nueva categoría que agregaste
    vida_normal=1095, # Vida útil sin abrir
    vida_refrigeracion=0,
    vida_congelacion=0
)
print(prueba_vino)